In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
HOPSWORKS_PROJECT_NAME = 'novusx10'  

In [3]:
import os
os.chdir("../")

In [4]:
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR/'.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [5]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):

    #download data for the whole year
    rides_one_year = load_raw_data(year)

    #append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
2023-11 file is not available
2023-12 file is not available


In [6]:
print(f'{len(rides)=}')

len(rides)=71247922


In [7]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:09<00:00, 27.53it/s]


In [8]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [9]:
import hopsworks

In [10]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value = HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/79009


In [11]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [12]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1   

In [13]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [28]:
feature_group.insert(ts_data, write_options={"wait_for_job":False})

Uploading Dataframe: 0.00% |          | Rows 0/4254840 | Elapsed Time: 00:00 | Remaining Time: ?

%4|1703106216.307|FAIL|rdkafka#producer-37| [thrd:ssl://18.225.98.177:9092/bootstrap]: ssl://18.225.98.177:9092/3: Disconnected (after 302312ms in state UP)
%4|1703106218.975|FAIL|rdkafka#producer-37| [thrd:ssl://18.227.183.195:9092/bootstrap]: ssl://18.227.183.195:9092/1: Disconnected (after 304970ms in state UP)
%4|1703106522.898|FAIL|rdkafka#producer-37| [thrd:ssl://18.225.98.177:9092/bootstrap]: ssl://18.225.98.177:9092/3: Disconnected (after 303379ms in state UP, 1 identical error(s) suppressed)
%6|1703106541.127|FAIL|rdkafka#producer-37| [thrd:ssl://18.227.183.195:9092/bootstrap]: ssl://18.227.183.195:9092/1: Disconnected (after 321126ms in state UP, 1 identical error(s) suppressed)


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/79009/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f9025fa5130>, None)

: 